In [1]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\combo_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")
print(loaded_combo_list)


Combo list loaded successfully.
['D:/data/imagenet\\n03450230\\', 'D:/data/imagenet\\n04273569\\', 'D:/data/imagenet\\n07836838\\', 'D:/data/imagenet\\n02510455\\', 'D:/data/imagenet\\n04146614\\', 'D:/data/imagenet\\n02085782\\', 'D:/data/imagenet\\n02328150\\', 'D:/data/imagenet\\n01491361\\', 'D:/data/imagenet\\n01776313\\', 'D:/data/imagenet\\n04392985\\', 'D:/data/imagenet\\n03590841\\', 'D:/data/imagenet\\n02667093\\', 'D:/data/imagenet\\n03344393\\', 'D:/data/imagenet\\n04525305\\', 'D:/data/imagenet\\n03769881\\', 'D:/data/imagenet\\n03459775\\', 'D:/data/imagenet\\n07831146\\', 'D:/data/imagenet\\n03197337\\', 'D:/data/imagenet\\n04591157\\', 'D:/data/imagenet\\n04069434\\', 'D:/data/imagenet\\n03393912\\', 'D:/data/imagenet\\n02018795\\', 'D:/data/imagenet\\n03692522\\', 'D:/data/imagenet\\n02100877\\', 'D:/data/imagenet\\n03494278\\', 'D:/data/imagenet\\n02009912\\', 'D:/data/imagenet\\n03127747\\', 'D:/data/imagenet\\n03180011\\', 'D:/data/imagenet\\n10148035\\', 'D:/data/i

In [2]:
len(loaded_combo_list)

300

In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the target size for resizing the images
target_size = (32, 32)

# Initialize empty lists to store resized images and labels
resized_images = []
labels = []

# Iterate over each directory in the loaded_combo_list
for directory in tqdm(loaded_combo_list[:2]):
    
    # Iterate over each file in the directory
    for filename in os.listdir(directory):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        
        # Open the image using PIL
        image = Image.open(file_path)
        
        # Convert the image to RGB if it's not already in RGB format
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Resize the image to the target size using PIL
        resized_image = image.resize(target_size)
        
        # Convert the image to a NumPy array
        resized_image = np.array(resized_image)
        
        # Append the resized image to the list
        resized_images.append(resized_image)
        
        # Append the corresponding label to the labels list
        labels.append(directory)

# Convert the list of resized images to a NumPy array
resized_images = np.array(resized_images)

# Normalize the pixel values between 0 and 1
resized_images = resized_images.astype('float32') / 255.0

# Convert labels to categorical using LabelEncoder and OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
onehot_encoder = OneHotEncoder(sparse=False)
categorical_labels = onehot_encoder.fit_transform(integer_encoded.reshape(-1, 1))

# Print the shape of the resized images and categorical labels arrays
print("Shape of resized images array:", resized_images.shape)
print("Shape of categorical labels array:", categorical_labels.shape)


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

In [10]:
labels[0]

'D:/data/imagenet\\n03450230\\'

In [12]:
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


# Load the pre-trained ResNet50 model (excluding the top layer)
base_model = ResNet50(weights=None, include_top=False, input_shape=(32, 32, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(512, activation='relu')(x)

# Add the output layer with the number of classes
output = Dense(categorical_labels.shape[1], activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)


In [14]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler

# Define the optimizer with SGD and momentum
optimizer = SGD(lr=0.1, momentum=0.9)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define a learning rate decay schedule
def learning_rate_decay(epoch):
    initial_lr = 0.1
    drop = 0.1
    epochs_drop = 10
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr

lr_scheduler = LearningRateScheduler(learning_rate_decay)

# Train the model
model.fit(resized_images, categorical_labels, batch_size=128, epochs=50, validation_split=0.2, callbacks=[lr_scheduler])

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/50
17/17 [==============================] - 10s 144ms/step - loss: 3.7733 - accuracy: 0.5851 - val_loss: 2257759905579607785472.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 2/50
17/17 [==============================] - 1s 67ms/step - loss: 5.4670 - accuracy: 0.5913 - val_loss: 1763471927734697984.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 3/50
17/17 [==============================] - 1s 68ms/step - loss: 9.3298 - accuracy: 0.6000 - val_loss: 225255964672.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 4/50
17/17 [==============================] - 1s 56ms/step - loss: 0.5114 - accuracy: 0.7615 - val_loss: 138568976.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 5/50
17/17 [==============================] - 1s 66ms/step - loss: 0.3649 - accuracy: 0.8548 - val_loss: 1233500.6250 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 6/50
17/17 [==============================] - 1s 63ms/step - loss: 0.3851 - accuracy: 0.8519 - val_loss: 89014.4844 - val_accuracy: 0.0000

KeyboardInterrupt: 